In [12]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
import glob

%load_ext autoreload
%autoreload 2
import sys
sys.path.append('/Users/tabea/Documents/UrbanMobility/src')
from models import transformer as transformer
from models import random_forest_model as random_forest

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
journey_16 = pd.read_csv('../data/processed/aggregated_journey_data/agg_journey_data_2016.csv', parse_dates=['end_date', 'start_date'], index_col=0)
journey_17 = pd.read_csv('../data/processed/aggregated_journey_data/agg_journey_data_2017.csv', parse_dates=['end_date', 'start_date'], index_col=0)
journey_18 = pd.read_csv('../data/processed/aggregated_journey_data/agg_journey_data_2018.csv', parse_dates=['end_date', 'start_date'], index_col=0)
journey_19 = pd.read_csv('../data/processed/aggregated_journey_data/agg_journey_data_2019.csv', parse_dates=['end_date', 'start_date'], index_col=0)

In [13]:
journey_train = pd.concat([journey_16, journey_17, journey_18])
journey_test = journey_19.copy()

x_train, y_train, x_test, y_test = random_forest.train_test_split(journey_train, journey_test)

In [83]:
n_boroughs = 12
num_sequences_train = len(x_train) // n_boroughs
num_sequences_test = len(x_test) // n_boroughs

# reshape input to be 3D [samples, timesteps, features]
x_train_reshaped = x_train.values[:num_sequences_train*n_boroughs].reshape((num_sequences_train, x_train.shape[1], n_boroughs))
x_test_reshaped = x_test.values[:num_sequences_test*n_boroughs].reshape((num_sequences_test, x_test.shape[1], n_boroughs))

# We don't reshape y, since it should be a 1D array
y_train_reshaped = y_train.values[:num_sequences_train]
y_test_reshaped = y_test.values[:num_sequences_test]


In [85]:
print(y_train_reshaped.shape, x_train_reshaped.shape)

(25450,) (25450, 104, 12)


In [87]:
## Testing the Transformer
tr = transformer.Transformer()
tr.train(x_train_reshaped,y_train_reshaped)


Epoch 1/3
796/796 [==============================] - 60s 73ms/step - loss: 13837.5742
Epoch 2/3
796/796 [==============================] - 59s 74ms/step - loss: 13642.7051
Epoch 3/3
796/796 [==============================] - 63s 79ms/step - loss: 13609.6504


In [88]:
from sklearn.metrics import mean_squared_error

# Predict outputs for the test set
y_pred = tr.predict(x_test_reshaped)

# Calculate the Mean Squared Error of the predictions
mse = mean_squared_error(y_test_reshaped, y_pred)

print(f'The Mean Squared Error of the predictions is {mse}')

266/266 [==============================] - 10s 36ms/step
The Mean Squared Error of the predictions is 14972.901400068036
